## Introduction
This notebook contains a few experiments with plotly color palettes and node coloring rules.



## Imports

In [1]:
import sys
from pathlib import Path
nb_dir = Path.cwd()
project_root = nb_dir if nb_dir.name == "idlmav" else nb_dir.parent
sys.path.append(str(project_root))

from idlmav.idlmav import available_renderers, plotly_renderer_context

In [2]:
import matplotlib.colors as mcolors
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots
import colorspacious as cs

## Select Plotly renderer


In [3]:
print(available_renderers)

['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode', 'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab', 'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg', 'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe', 'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']


In [4]:
renderer = 'notebook_connected'

## Built-in palettes

### Discrete colors
[Reference](https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express)

In [5]:
with plotly_renderer_context(renderer):
    fig = pc.qualitative.swatches()
    fig.show()

### Continuous colors (sequential)
[Reference](https://plotly.com/python/builtin-colorscales/#builtin-sequential-color-scales)

In [6]:
with plotly_renderer_context(renderer):
    fig = pc.sequential.swatches_continuous()
    fig.show()

### Continuous colors (diverging)
[Reference](https://plotly.com/python/builtin-colorscales/#builtin-diverging-color-scales)

In [7]:
with plotly_renderer_context(renderer):
    fig = pc.diverging.swatches_continuous()
    fig.show()

### Cyclical colors
[Reference](https://plotly.com/python/builtin-colorscales/#builtin-cyclical-color-scales)

In [8]:
with plotly_renderer_context(renderer):
    fig = pc.cyclical.swatches_cyclical()
    fig.show()

## Palette queries and conversions

In [9]:
def rgb_to_hex(color):
    def convert_single(color_str):
        if isinstance(color_str, str) and color_str.startswith("rgb"):
            r, g, b = map(int, color_str[4:-1].split(","))
            return f"#{r:02X}{g:02X}{b:02X}"
        return color_str

    if isinstance(color, (list, tuple)):
        return type(color)(convert_single(c) for c in color)
    else:
        return convert_single(color)

In [10]:
def to_hex(colors):
    return rgb_to_hex(pc.convert_colors_to_same_type(colors, colortype="rgb")[0])

In [11]:
def colors_in_palette(palette):
    return to_hex(getattr(pc.qualitative, palette))

In [12]:
print(to_hex(pc.qualitative.Bold))

['#7F3C8D', '#11A579', '#3969AC', '#F2B701', '#E73F74', '#80BA5A', '#E68310', '#008695', '#CF1C90', '#F97B72', '#A5AA99']


In [13]:
print(to_hex(pc.qualitative.Dark24))

['#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D', '#DA16FF', '#222A2A', '#B68100', '#750D86', '#EB663B', '#511CFB', '#00A08B', '#FB00D1', '#FC0080', '#B2828D', '#6C7C32', '#778AAE', '#862A16', '#A777F1', '#620042', '#1616A7', '#DA60CA', '#6C4516', '#0D2A63', '#AF0038']


In [14]:
qualitative_colors = [attr for attr in dir(pc.qualitative) if not attr.startswith("_")]
print(qualitative_colors)

['Alphabet', 'Alphabet_r', 'Antique', 'Antique_r', 'Bold', 'Bold_r', 'D3', 'D3_r', 'Dark2', 'Dark24', 'Dark24_r', 'Dark2_r', 'G10', 'G10_r', 'Light24', 'Light24_r', 'Pastel', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'Pastel_r', 'Plotly', 'Plotly_r', 'Prism', 'Prism_r', 'Safe', 'Safe_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'T10', 'T10_r', 'Vivid', 'Vivid_r', 'swatches']


In [15]:
print(colors_in_palette('Vivid'))

['#E58606', '#5D69B1', '#52BCA3', '#99C945', '#CC61B0', '#24796C', '#DAA51B', '#2F8AC4', '#764E9F', '#ED645A', '#A5AA99']


## Proximity of pairs of discrete colors

In [16]:
def hex_to_rgb(hex_color):
    """Convert a hex color string (#RRGGBB) to an RGB tuple (R, G, B)."""
    hex_color = hex_color.lstrip("#")
    return tuple(int(hex_color[i:i + 2], 16) / 255.0 for i in (0, 2, 4))

def rgb_to_lab(rgb_color):
    """Convert an RGB tuple to CIELAB using colorspacious."""
    return cs.cspace_convert(rgb_color, "sRGB1", "CIELab")

def rgb_to_hsv(rgb_color):
    """Convert an RGB tuple to HSV using matplotlib."""
    return mcolors.rgb_to_hsv(rgb_color)

def visualize_cielab_colors(hex_colors):
    """Visualize a list of colors in both CIELAB and HSV spaces using Plotly subplots."""
    rgb_colors = [hex_to_rgb(color) for color in hex_colors]
    lab_colors = [rgb_to_lab(color) for color in rgb_colors]
    hsv_colors = [rgb_to_hsv(color) for color in rgb_colors]

    # Extract L*, a*, b*, H, S, and V values
    l_values = [lab[0] for lab in lab_colors]
    a_values = [lab[1] for lab in lab_colors]
    b_values = [lab[2] for lab in lab_colors]
    h_values = [hsv[0] for hsv in hsv_colors]
    s_values = [hsv[1] for hsv in hsv_colors]
    v_values = [hsv[2] for hsv in hsv_colors]

    # Create subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=["CIELAB Space", "HSV Space"])

    for idx, (hex_color, a, b, l, h, s, v) in enumerate(zip(hex_colors, a_values, b_values, l_values, h_values, s_values, v_values)):
        lab_scatter = go.Scatter(
            x=[a],
            y=[b],
            mode="markers",
            marker=dict(size=10, color=hex_color),
            text=[f"Index: {idx}<br>Hex: {hex_color}<br>L*: {l:.2f}<br>a*: {a:.2f}<br>b*: {b:.2f}"],
            name=f"{idx}: {hex_color}",
            hoverinfo="text",
            legendgroup=f"group{idx}"
        )
        fig.add_trace(lab_scatter, row=1, col=1)
        
    for idx, (hex_color, a, b, l, h, s, v) in enumerate(zip(hex_colors, a_values, b_values, l_values, h_values, s_values, v_values)):
        hsv_scatter = go.Scatter(
            x=[s],
            y=[v],
            mode="markers",
            marker=dict(size=10, color=hex_color),
            text=[f"Index: {idx}<br>Hex: {hex_color}<br>H: {h:.2f}<br>S: {s:.2f}<br>V: {v:.2f}"],
            name=f"{idx}: {hex_color}",
            hoverinfo="text",
            legendgroup=f"group{idx}",
            showlegend=False
        )
        fig.add_trace(hsv_scatter, row=1, col=2)

    fig.update_xaxes(title_text="a* (Green-Red)", row=1, col=1, scaleanchor="y1", showgrid=False, zeroline=False)
    fig.update_yaxes(title_text="b* (Blue-Yellow)", row=1, col=1, showgrid=False, zeroline=False)
    fig.update_xaxes(title_text="Saturation", row=1, col=2, scaleanchor="y2", showgrid=False, zeroline=False)
    fig.update_yaxes(title_text="Value", row=1, col=2, showgrid=False, zeroline=False)

    fig.update_layout(
        legend=dict(orientation="h",y=-0.3)
    )

    fig.show()

In [17]:
with plotly_renderer_context(renderer):
    dark24_colors = to_hex(pc.qualitative.Dark24)
    visualize_cielab_colors(dark24_colors)

* Skip color 1 (#E15F99): it is perceptually close to color 20 (#DA60CA)
* Skip color 11 (#FB00D1): it is perceptually between similar colors 4 (#DA16FF), 12 (#FC0080) and 20 (#DA60CA)
* Use color 15 (#778AAE) for the "everything else" category